In [ ]:
import requests as r

In [ ]:
url ="https://api.covid19api.com/dayone/country/brazil"

In [7]:
resp = r.get(url)
print(resp.status_code)

200


In [10]:
raw_data = resp.json()
print(raw_data[0])

{'ID': '63aa488e-b4a2-4fcd-ae15-07c1ff219903', 'Country': 'Brazil', 'CountryCode': 'BR', 'Province': '', 'City': '', 'CityCode': '', 'Lat': '-14.24', 'Lon': '-51.93', 'Confirmed': 1, 'Deaths': 0, 'Recovered': 0, 'Active': 1, 'Date': '2020-02-26T00:00:00Z'}


In [12]:
final_data = []
for obs in raw_data:
    final_data.append([obs['Confirmed'], obs['Deaths'], obs['Recovered'], obs['Active'], obs['Date']])

print(final_data[0])

[1, 0, 0, 1, '2020-02-26T00:00:00Z']


In [17]:
final_data.insert(0, ['Confirmados', 'Obitos', 'Recuperados', 'Ativos', 'Data'])
print(final_data[0])

['Confirmados', ' Mortos', 'Recuperados', 'Ativos', 'Data']


In [24]:
confirmados=0
obitos=1
recuperados=2
ativos=3
data=4

for i in range(1, len(final_data)):
    final_data[i][data] = final_data[i][data][:10]

print(final_data[1])

[1, 0, 0, 1, '2020-02-26']


In [ ]:
import datetime as dt